# Generate Unit Tests for SQLModels from Template

In [1]:
# Import packages
import os
import yaml

## Set the Output directory

In [2]:
# Get the current working directory
current_dir = os.getcwd()

# Remove the last folder from the current working directory
parent_dir = os.path.dirname(current_dir)

# Define the output folder
output_folder = os.path.join(parent_dir, "models")

# Try and except block to check if the folder exists
try:
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created directory: {output_folder}")
    else:
        print(f"Directory already exists: {output_folder}")
except Exception as e:
    print(f"An error occurred: {e}")

Directory already exists: /workspaces/coaching-companion/tests/models


## List of Models to Generate Unit Tests
All the models will be based on the BaseTableModel, and you will need to customize the tests for each model.

In [3]:
# FUNCTION: Read the model data from the yaml file
def load_yaml(file_path):
    with open(file_path, "r") as file:
        data = yaml.safe_load(file)
    return data

In [4]:
# Load the model data
model_data = load_yaml(os.path.join(os.getcwd(), "models_2_template2.yml"))
print(model_data["models"])

['AWSDashboard', 'Access', 'AccreditationCertificate', 'ActionPlans', 'AddPlayByPlay', 'Alias', 'Annotations', 'AssessmentsClassroom', 'AssessmentsDashboard', 'AssessmentsEvaluation', 'AssessmentsLister', 'AssessmentsRating', 'AssessmentsReport', 'AssessmentsReportClassroom', 'AssessmentsSite', 'Assets', 'Assignments', 'AuthUser', 'BaseTableModel', 'CoachingPartnership', 'CollaboraEndpoint', 'Comments', 'ConnectToMerit', 'ContactForm', 'CopAssets', 'CopGroups', 'CopLocation', 'CoursesState', 'CycleDashboard', 'DiscussionBoards', 'DiscussionTopics', 'Documentation', 'DualLanguageHandbook', 'DualLanguageSubmission', 'EaEvalSurvey', 'Endpoint', 'FamilySurveyLink', 'Grade', 'Inquisitor', 'InstructorMaterials', 'Item', 'Layout', 'Limit', 'Links', 'Lister', 'Location', 'Login', 'Logout', 'LtiEndpoint', 'ManageComponents', 'ManageDatalists', 'ManageGroups', 'ManageMenus', 'ManageRecipes', 'ManageSite', 'ManageStudents', 'ManageUsers', 'Media', 'MediaDatabase', 'MeritEndpoint', 'MssqlServer', 

## Define the Model and the Test Template

In [5]:
# FUNCTION: Open a file and write the model template
def write_model_template(output_folder, model_name, model_template):
    output_filepath = os.path.join(output_folder,f"test_{model_name}.py")
    with open(output_filepath, "w") as file:
        file.write(f"{model_template}")

In [6]:
for model in model_data["models"]:
    model_template = \
f"""import pytest
from datetime import datetime, timezone
from coaching_companion.models import {model}

# Define the datetime string
datetime_str = "2021-12-01T00:00:00Z"
# Parse the datetime string into a datetime object
datetime_obj = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%SZ")
# Convert the datetime object to a Unix timestamp (float)
unix_timestamp = datetime_obj.replace(tzinfo=timezone.utc).timestamp()

# Define the url string
url_str = "https://s3-us-west-2.amazonaws.com/test-bucket/test-key"

# Define the UUID string
uuid_int = 12345678901234567890

@pytest.mark.parametrize("unix_timestamp", [unix_timestamp]) # Allows us to define a single test with multiple potential inputs
def test_{model.lower()}(unix_timestamp):
    # Create an instance of {model}
    dashboard = {model}.model_validate(
        {{"created_at": unix_timestamp,
        "created_by": int(12345678901234567890),
        "title": "Test {model} Title",
        "type_": "{model.lower()}"}}
    )

    # Assert that the fields are correctly set
    assert dashboard.created_by == int(12345678901234567890)
    assert dashboard.title == "Test {model} Title"
    assert dashboard.type_ == "{model.lower()}"

    # Convert the Unix timestamp to a UTC datetime object
    expected_created_at = datetime.fromtimestamp(unix_timestamp, tz=timezone.utc)
    # Format the datetime object to the desired string format
    expected_created_at_str = expected_created_at.strftime("%Y-%m-%dT%H:%M:%SZ")

    # Assert that the created_at field is correctly converted and formatted
    assert expected_created_at_str == datetime_str

def test_{model.lower()}_default_values():
    # Create an instance of {model} without optional fields
    dashboard = {model}(title="Test Dashboard")

    # Assert that the default values are correctly set
    assert dashboard.id is None
    assert dashboard.created_by is None
    assert dashboard.created_at is None

def test_{model.lower()}_name_max_length():
    # Create an instance of {model} with a name exceeding max_length
    long_name = "A" * 300 # For testing non-text type fields
    long_num = 12345678901234567890 # For testing text type fields
    # Validate the id
    with pytest.raises(ValueError):
        {model}.model_validate({{"id": long_name}})
    # Validate the title
    with pytest.raises(ValueError):
        {model}.model_validate({{"title": long_name}})
    # Validate the created_by
    with pytest.raises(ValueError):
        {model}.model_validate({{"created_by": long_name}})
    # Validate the created_at
    with pytest.raises(OverflowError):
        {model}.model_validate({{"created_at": long_num}})
    # Validate the created_at
    with pytest.raises(TypeError):
        {model}.model_validate({{"created_at": long_name}})
    # Validate the type_
    with pytest.raises(ValueError):
        {model}.model_validate({{"type_": long_name}})

# Run the tests
if __name__ == "__main__":
    pytest.main()
    
# References:
# - https://github.com/fastapi/sqlmodel/issues/52
# - https://www.datacamp.com/tutorial/pytest-tutorial-a-hands-on-guide-to-unit-testing
    """
    
    write_model_template(output_folder=output_folder, model_name=model.lower(), model_template=model_template)